In [1]:
# 참고 사이트 
# https://notebook.community/zzsza/Datascience_School/29.%20%EC%B6%94%EC%B2%9C%20%EC%8B%9C%EC%8A%A4%ED%85%9C/01.%20%EC%B6%94%EC%B2%9C%20%EC%8B%9C%EC%8A%A4%ED%85%9C%EC%9D%98%20%EA%B8%B0%EC%B4%88
#  유저가 부여한 도서 평점을 회귀 예측하는 AI 모델을 개발해야 합니다.

In [1]:
!unzip '/content/drive/MyDrive/Dacon/open.zip'

Archive:  /content/drive/MyDrive/Dacon/open.zip
replace sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [2]:
import pandas as pd
import numpy as np

In [19]:
!pip install scikit-surprise
import pandas as pd
from surprise import SVD, Dataset, Reader, accuracy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

In [6]:
train.head()

,ID,User-ID,Book-ID,Book-Rating,Age,Location,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,TRAIN_000000,USER_00000,BOOK_044368,8,23.0,"sackville, new brunswick, canada",Road Taken,Rona Jaffe,2001.0,Mira
1,TRAIN_000001,USER_00000,BOOK_081205,8,23.0,"sackville, new brunswick, canada",Macbeth (New Penguin Shakespeare),William Shakespeare,1981.0,Penguin Books
2,TRAIN_000002,USER_00000,BOOK_086781,0,23.0,"sackville, new brunswick, canada",Waverley (Penguin English Library),Walter Scott,1981.0,Penguin Books
3,TRAIN_000003,USER_00000,BOOK_098622,0,23.0,"sackville, new brunswick, canada",Mother Earth Father Sky,Sue Harrison,1991.0,Avon
4,TRAIN_000004,USER_00000,BOOK_180810,8,23.0,"sackville, new brunswick, canada",She Who Remembers,Linda Lay Shuler,1989.0,Signet Book


In [7]:
# Surprise 라이브러리용 Reader 및 Dataset 객체 생성
reader = Reader(rating_scale=(0, 10))
train = Dataset.load_from_df(train[['User-ID', 'Book-ID', 'Book-Rating']], reader)
train = train.build_full_trainset()

In [8]:
# SVD 모델 훈련
model = SVD()
model.fit(train)

In [13]:
submit = pd.read_csv('./sample_submission.csv')

In [14]:
submit['Book-Rating'] = test.apply(lambda row: model.predict(row['User-ID'], row['Book-ID']).est, axis=1)

In [ ]:
submit.to_csv('./baseline_submit.csv', index=False)

개선


In [23]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

In [24]:
reader = Reader(rating_scale=(0, 10))
train = Dataset.load_from_df(train[['User-ID', 'Book-ID', 'Book-Rating']], reader)
train = train.build_full_trainset()

In [25]:
# 하이퍼 파라메터 튜닝
from surprise.model_selection import GridSearchCV
param_grid = {
    # 'n_factors' :[50,100,150],
    # 'n_epochs' :[20,30,40],
    # 'lr_all' :[0.002,0.005,0.008],
    # 'reg_all' :[0.02,0.04,0.06]
    'n_epochs' :[5,10],
    'lr_all' :[0.002,0.005],
    'reg_all' :[0.02,0.04]

}

In [26]:
from surprise.model_selection import train_test_split
train = pd.read_csv('./train.csv')
train = Dataset.load_from_df(train[['User-ID', 'Book-ID', 'Book-Rating']], reader)
gs = GridSearchCV(SVD,param_grid, cv=2, measures=['rmse'],joblib_verbose=5)
gs.fit(train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    8.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   15.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   23.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:   31.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed:  2.3min finished


In [27]:
gs.best_score

{'rmse': 3.457724733834722}

In [28]:
gs.best_params

{'rmse': {'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.04}}

In [40]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')
reader = Reader(rating_scale=(0, 10))
train = Dataset.load_from_df(train[['User-ID', 'Book-ID', 'Book-Rating']], reader)
train = train.build_full_trainset()
model = SVD(n_epochs=10,lr_all=0.005,reg_all=0.04)
model.fit(train)
submit['Book-Rating'] = test.apply(lambda row: model.predict(row['User-ID'], row['Book-ID']).est, axis=1)
submit.to_csv('./baseline_submit_gridsearch.csv', index=False)

In [30]:
best_model = gs.best_estimator

In [39]:
test = pd.read_csv('./test.csv')
best_model_gs =  best_model['rmse']
submit['Book-Rating'] = test.apply(lambda row: best_model_gs.predict(row['User-ID'], row['Book-ID']).est, axis=1)
submit.to_csv('./baseline_submit_gridsearch.csv', index=False)

AttributeError: ignored

In [32]:
model

In [35]:
best_model['rmse']

In [6]:
import pandas as pd
import numpy as np
# 재 도전
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')
train_df = train[['User-ID', 'Book-ID', 'Book-Rating']]

# user_id 와 book_id를 숫자로 변환 - 고유한 값으로 
user_dict = {user_id: idx for idx , user_id in enumerate( train_df['User-ID'].unique())}
book_dict = {book_id: idx for idx , book_id in enumerate( train_df['Book-ID'].unique())}
train_df['User-ID'] = train_df['User-ID'].apply(lambda x : user_dict[x])
train_df['Book-ID'] = train_df['Book-ID'].apply(lambda x : book_dict[x])
train_df.head()

<ipython-input-6-132b9bf96086>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['User-ID'] = train_df['User-ID'].apply(lambda x : user_dict[x])
<ipython-input-6-132b9bf96086>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['Book-ID'] = train_df['Book-ID'].apply(lambda x : book_dict[x])


,User-ID,Book-ID,Book-Rating
0,0,0,8
1,0,1,8
2,0,2,0
3,0,3,0
4,0,4,8


In [8]:
train_df['Book-Rating'].unique()

array([ 8,  0,  5,  9,  7,  6, 10,  2,  3,  4,  1])

In [ ]:
data = train_df[train_df['Book-Rating'] != 0]
data['Book-Rating'] = pd.Categorical(data['Book-Rating'], categories = [1,2,3,4,5],ordered=True)
data['Book-Rating'] = data['Book-Rating'].cat.codes+1

In [ ]:
data = data.drop_duplicates()
data.info()

In [ ]:
from scipy.sparse import coo_matrix
rarings = coo_matrix( (data['Book-Rating'],( data['User-ID'],data['Book-ID']  )  ))
rarings

In [20]:
svd = SVD()

In [ ]:
#  userid와 bookid 만 정수형으로 전처리해서 적용
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')
reader = Reader(rating_scale=(0, 10))
train = Dataset.load_from_df(train[['User-ID', 'Book-ID', 'Book-Rating']], reader)
train = train.build_full_trainset()
model = SVD(n_epochs=10,lr_all=0.005,reg_all=0.04)
model.fit(train)
submit['Book-Rating'] = test.apply(lambda row: model.predict(row['User-ID'], row['Book-ID']).est, axis=1)
submit.to_csv('./baseline_submit_gridsearch.csv', index=False)

In [ ]:
# 하이퍼 파라메터 튜닝
from surprise.model_selection import GridSearchCV
param_grid = {
    # 'n_factors' :[50,100,150],
    # 'n_epochs' :[20,30,40],
    # 'lr_all' :[0.002,0.005,0.008],
    # 'reg_all' :[0.02,0.04,0.06]
    'n_epochs' :[5,10],
    'lr_all' :[0.002,0.005],
    'reg_all' :[0.02,0.04]
}

train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

user_dict = {user_id: idx for idx , user_id in enumerate( train['User-ID'].unique())}
book_dict = {book_id: idx for idx , book_id in enumerate( train['Book-ID'].unique())}
train['User-ID'] = train['User-ID'].apply(lambda x : user_dict[x])
train['Book-ID'] = train['Book-ID'].apply(lambda x : book_dict[x])

reader = Reader(rating_scale=(0, 10))
train = Dataset.load_from_df(train[['User-ID', 'Book-ID', 'Book-Rating']], reader)
gs = GridSearchCV(SVD,param_grid, cv=3, measures=['rmse'],joblib_verbose=5)
gs.fit(train)